# SMILearn models
Evaluation of binary biological activity prediction models based on SMILearn matrices

## Prerequisites

### Installations (Google Colab)
This is an example of installation steps that allow this notebook to work properly in Google Colab environment.  
In other cases, it is recommended to follow the instructions in `README.md` file and skip the cell below.

In [ ]:
# Save the output as `installation_log` variable
%%capture installation_log

# Install Miniconda
!wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
!rm Miniconda3-latest-Linux-x86_64.sh

# Install RDKit
!conda install -qyc conda-forge rdkit

# Install DeepSMILES unofficial GitHub fork
!pip install git+git://github.com/mateuszrezler/deepsmiles@master

# Download sample data set
!wget http://www.dna.bio.keio.ac.jp/smiles/TOX21/\
NR-PPAR-gamma_wholetraining.smiles

# Update system path
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages')

### Imports

In [ ]:
import deepsmiles
import numpy as np
import pandas as pd
import re
from rdkit import Chem
from rdkit.Chem import rdchem

## Global settings

### Constants

In [ ]:
BOND_CHARS = '.-/\\:=#$'
BOND_GEOMETRY_DICT = {'.': 0, '-': 0, ':': 0, '=': 0, '#': 0, '$': 0,
                      '/': 1, '\\': 2}
BOND_ORDER_DICT = {'-': 1.0, '/': 1.0, '\\': 1.0,
                   '.': 0.5, ':': 1.5, '=': 2.0, '#': 3.0, '$': 4.0}
CHIRAL_TAGS = ['', '@', '@@']
FEATURE_MATRIX_COLUMN_NAMES = ['Atomic number', 'Aromaticity', 'Chirality',
                               'Total-H-count', 'Degree', 'Ring membership',
                               'Ring size', 'Valence', 'Connectivity',
                               'Charge', 'Bond order', 'Bond geometry',
                               'Branch length', 'Ring size']
FULL_SMARTS_TEMPLATE = '[{}{}H{}D{}R{}r{}v{}X{}{}]\n'
PARENTHESIS_DICT = {'(': 1, ')': -1}
SIGN_VISIBLE = '{0:+d}'
SMILES_REGEX_PART_1 = r'(?:\[.*?\])|[\-/\\:=#\$\.]|%\d{2'
SMILES_REGEX_PART_2 = r'}|\d|[\(\)]'
SMILES_REGEX_PART_2_DEEP = r',}|\d|\)+'

### Classes and functions

In [ ]:
# lambdas
abs_if_true = lambda value, test: np.abs(value) if test else value
aromatic = lambda atom: int(atom.GetIsAromatic())
at_num = lambda atom: atom.GetAtomicNum()
bond_order = lambda bond: BOND_ORDER_DICT[bond]
bond_geometry = lambda bond: BOND_GEOMETRY_DICT[bond]
charge = lambda atom: atom.GetFormalCharge()
charge_tag = lambda atom: \
    re.sub('([\+\-])1', r'\1', SIGN_VISIBLE.format(charge(atom))) \
    if charge(atom) else ''
chiral = lambda atom: int(atom.GetChiralTag())
chiral_tag = lambda atom: CHIRAL_TAGS[chiral(atom)]
D = lambda atom: atom.GetDegree()
H = lambda atom: atom.GetTotalNumHs()
normalize_to_largest = lambda matrix, largest_len: \
    np.vstack((matrix, np.zeros((largest_len-matrix.shape[0], matrix.shape[1]),
                                dtype=float)))
parenthesis_count = lambda chars: \
    sum([PARENTHESIS_DICT[char] for char in chars])
pop_start_char = lambda text, char: text[1:] if text[0] == char else text
R = lambda smallest_rings: len(smallest_rings)
r = lambda smallest_rings: \
    min(smallest_rings) if len(smallest_rings) > 0 else 0
rebuild_smiles = lambda smiles, canonical, **properties: \
    Chem.MolToSmiles(Chem.MolFromSmiles(smiles), canonical=canonical,
                     **properties)
series_to_3d_array = lambda series: np.moveaxis(np.dstack(series), 2, 0)
symbol = lambda atom: \
    atom.GetSymbol().lower() if atom.GetIsAromatic() else atom.GetSymbol()
v = lambda atom: atom.GetTotalValence()
X = lambda atom: atom.GetTotalDegree()

In [ ]:
class SMIList(list):

    """SMILES as list of rdchem.Atom objects and other chars."""

    def __init__(self, smiles, canonical=True, deep=True, **properties):

        self.__smiles = rebuild_smiles(smiles, canonical, **properties)
        self.__canonical = canonical
        self.__deep = deep

        if deep:
            self.__activate_deep_converter()

        self.__properties = properties
        self.__build()

    def __repr__(self):

        return str([f'<At.{component.GetSymbol()}>'
                    if type(component) == rdchem.Atom else component
                    for component in self])

    def __str__(self):

        rebuilt = rebuild_smiles(self.__smiles, canonical=self.__canonical,
                                 **self.__properties)

        return self.__deep_or_not(rebuilt)

    def __activate_deep_converter(self):

        self.__deep_converter = deepsmiles.Converter(branches=True, rings=True)

    @staticmethod
    def __atom_features(atom, functions, atom_index=None, ring_info=None):

        target_dict = {'atom': atom}

        if 'ring' in functions['targets']:
            smallest_rings = [len(atom_tuple) for atom_tuple in ring_info
                              if atom_index in atom_tuple]
            target_dict.update({'ring': smallest_rings})

        features = [function(target_dict[functions['targets'][index]])
                    for index, function_list
                    in enumerate(functions['groups'])
                    for function in function_list]

        return features

    def __build(self):

        self.clear()
        mol = Chem.MolFromSmiles(self.__smiles)
        atoms = [rdchem.Mol.GetAtomWithIdx(mol, atom_index)
                 for atom_index in range(mol.GetNumAtoms())]
        rebuilt = rebuild_smiles(self.__smiles, canonical=False,
                                 allBondsExplicit=True, allHsExplicit=True)
        explicit = self.__deep_or_not(rebuilt)

        if self.__deep:
            part_2 = SMILES_REGEX_PART_2_DEEP

        else:
            part_2 = SMILES_REGEX_PART_2

        pattern = SMILES_REGEX_PART_1 + part_2
        components = re.findall(pattern, explicit)
        atom_index = 0

        for component in components:

            if component[0] == '[':
                self.append(atoms[atom_index])
                atom_index += 1

            else:
                self.append(component)

    def __deep_or_not(self, smiles):

        return self.__deep_converter.encode(smiles) if self.__deep else smiles

    @staticmethod
    def __features(component, functions):

        return [function(component) for function in functions]

    @property
    def canonical(self):

        return self.__canonical

    @canonical.setter
    def canonical(self, setting):

        self.__canonical = setting
        self.__smiles = rebuild_smiles(self.__smiles, setting)
        self.__build()

    @property
    def deep(self):

        return self.__deep

    @deep.setter
    def deep(self, setting):

        if setting:
            self.__activate_deep_converter()

        self.__deep = setting
        self.__build()

    @property
    def properties(self):

        return self.__properties

    @properties.setter
    def properties(self, setting):

        self.__properties = setting
        self.__build()

    @property
    def smiles(self):

        return self.__smiles

    @smiles.setter
    def smiles(self, setting):

        self.__smiles = setting
        self.__build()

    @property
    def str(self):

        return self.__str__()

    def to_full_smarts(self):

        """Convert SMIList to fullSMARTS string."""

        atom_functions = {'targets': ('atom', 'ring', 'atom'),
                          'groups': ((symbol, chiral_tag, H, D),
                                     (R, r),
                                     (v, X, charge_tag))}
        atom_index = 0
        full_smarts = ''
        mol = Chem.MolFromSmiles(self.__smiles)
        ring_info = mol.GetRingInfo().AtomRings()

        for component_index, component in enumerate(self):

            if type(component) == rdchem.Atom:
                atom_features = self.__atom_features(component, atom_functions,
                                                     atom_index, ring_info)
                full_smarts += FULL_SMARTS_TEMPLATE.format(*atom_features)
                atom_index += 1

            else:
                full_smarts += component + '\n'

        return full_smarts

    def to_matrix(self,
                  atom_functions={'targets': ('atom', 'ring', 'atom'),
                                  'groups': ((at_num, aromatic, chiral, H, D),
                                             (R, r),
                                             (v, X, charge))},
                  bond_functions=(bond_order, bond_geometry),
                  branch_functions=(parenthesis_count,),
                  ring_functions=(lambda x: pop_start_char(x, '%'),)):

        """Convert SMIList to customizable feature matrix (numpy.ndarray)."""

        atom_index = 0
        mol = Chem.MolFromSmiles(self.__smiles)
        ring_info = mol.GetRingInfo().AtomRings()

        for component_index, component in enumerate(self):

            if atom_functions and type(component) == rdchem.Atom:
                atom_features = self.__atom_features(component, atom_functions,
                                                     atom_index, ring_info)
                all_features = np.hstack((np.array(atom_features, dtype=float),
                                          np.zeros(4, dtype=float)))
                atom_index += 1

            elif bond_functions and component in BOND_CHARS:
                bond_features = self.__features(component, bond_functions)
                arrays = (np.zeros(len(atom_features), dtype=float),
                          np.array(bond_features, dtype=float),
                          np.zeros(2, dtype=float))
                all_features = np.hstack(arrays)

            elif branch_functions and component[0] in PARENTHESIS_DICT:
                branch_features = self.__features(component, branch_functions)
                arrays = (np.zeros(len(atom_features)+2, dtype=float),
                          np.array(branch_features, dtype=float),
                          np.zeros(1, dtype=float))
                all_features = np.hstack(arrays)

            elif ring_functions \
                    and (component[0].isdigit() or component[0] == '%'):
                ring_features = self.__features(component, ring_functions)
                arrays = (np.zeros(len(atom_features)+3, dtype=float),
                          np.array(ring_features, dtype=float))
                all_features = np.hstack(arrays)

            if component_index == 0:
                feature_matrix = all_features

            else:
                feature_matrix = np.vstack((feature_matrix, all_features))

        return feature_matrix


# DEMO
acetone = 'CC(=O)C'
smi = SMIList(acetone, canonical=True, deep=True, allBondsExplicit=True)
print(smi.to_full_smarts())
print(smi.to_matrix().shape)
smi.to_matrix()

## Main program

### Data preparation

In [ ]:
# load data set
data = pd.read_csv('NR-PPAR-gamma_wholetraining.smiles', delimiter=' ',
                   header=None, names=['SMILES', 'Activity'])

# calculate 2D feature matrix (rows: atoms, columns: atom features)
data['Feature matrix'] = data['SMILES'].apply(lambda x: SMIList(x).to_matrix())

# normalize all matrices
largest_matrix_len = data['Feature matrix'].str.len().max()
data['Feature matrix'] = data['Feature matrix'].apply(
    lambda x: normalize_to_largest(x, largest_matrix_len))

In [ ]:
data

In [ ]:
# quick preview of selected array
print('Full SMARTS:', SMIList(data['SMILES'][1]).to_full_smarts(), sep='\n')
pd.DataFrame(data['Feature matrix'][1], columns=FEATURE_MATRIX_COLUMN_NAMES)

In [ ]:
X = series_to_3d_array(data['Feature matrix'])
X.shape